In [11]:
!git clone https://github.com/kenhding/sleepClassifier.git
%cd /content/sleepClassifier
!cp /content/drive/MyDrive/Colab Notebooks/sleep_classification_2024.ipynb /content/sleepClassifier/

Cloning into 'sleepClassifier'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.
/content/sleepClassifier
cp: cannot stat '/content/drive/MyDrive/Colab': No such file or directory
cp: cannot stat 'Notebooks/sleep_classification_2024.ipynb': No such file or directory


In [ ]:
!pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 74.2 MB/s eta 0:00:00


In [ ]:
import os
import mne

edfpath = '/content/drive/MyDrive/unimedizinSleepEEGproject/shareLink/edf/'

mainz_edf = os.listdir(edfpath)
print(mainz_edf)

edf_item = mne.io.read_raw_edf(os.path.join(edfpath,mainz_edf[0]))
edf_item



['BERIR003_22.01.2021_14-04-03.edf', 'BOTDO001_22.01.2021_14-05-05.edf', 'BRAJE000_22.01.2021_13-51-49.edf', 'BANSI000_22.01.2021_13-46-47.edf', 'BERIR001_22.01.2021_14-03-06.edf', 'BAUHA000_22.01.2021_14-12-16.edf', 'BOTDO000_22.01.2021_14-02-04.edf', 'BANSI001_22.01.2021_13-44-43.edf', 'AL_HA000_22.01.2021_12-33-56.edf', 'AHMMA000_22.01.2021_12-50-34.edf', 'AHMMA001_22.01.2021_12-52-51.edf', 'DZIPI001_22.01.2021_13-40-39.edf', 'CHRHE002_22.01.2021_13-59-53.edf', 'FUCRA000_22.01.2021_13-07-26.edf', 'FRAVI000_22.01.2021_13-55-56.edf', 'FREPE000_22.01.2021_13-11-43.edf', 'DZIPI000_22.01.2021_13-37-26.edf', 'CHRFA001_22.01.2021_14-08-04.edf', 'CHRFA000_22.01.2021_14-06-08.edf', 'BUCHA000_22.01.2021_12-43-43.edf', 'CHRHE001_22.01.2021_13-58-57.edf', 'BUSHA001_22.01.2021_12-31-21.edf', 'HAGEL001_22.01.2021_13-53-57.edf', 'HAGTH000_22.01.2021_13-31-09.edf', 'HAGTH003_22.01.2021_13-32-07.edf', 'HAMAN000_22.01.2021_13-45-44.edf', 'GRYVA000_22.01.2021_13-38-33.edf', 'GLARO003_22.01.2021_13-57-

<RawEDF | BERIR003_22.01.2021_14-04-03.edf, 26 x 5900000 (29500.0 s), ~30 kB, data not loaded>

In [ ]:
edf_item.info

<Info | 8 non-empty values
 bads: []
 ch_names: Resp nasal, Resp chest, Sound SPL, Pulsewave, Resp abdomen, ...
 chs: 26 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 100.0 Hz
 meas_date: 2019-01-05 21:47:26 UTC
 nchan: 26
 projs: []
 sfreq: 200.0 Hz
 subject_info: 3 items (dict)
>

In [ ]:
# Predefined mapping: standard channel names mapped to possible variations
channel_mapping = {
    'EEG F4-M1': ['EEG F4-M1', 'F4-A1'],
    'EEG C4-M1': ['EEG C4-M1', 'C4-A1', 'EEG C4/A1', 'C4-A1*'],
    'EEG O2-M1': ['EEG O2-M1', 'O2-A1', 'EEG O2/A1'],
    'EEG C3-M2': ['EEG C3-M2', 'C3-A2', 'EEG C3/A2', 'EEG C3/A2*'],
    # EOG channels with variations
    'EOG E1-M2': ['EOG E1-M2', 'LOC-A2', 'EOG left'],
    'EOG E2-M2': ['EOG E2-M2', 'ROC-A1', 'EOG right'],
    # ECG channels with variations
    'ECG': ['ECG', 'X2']
}

ordered_channels = []
# Find and rename channels based on the mapping
for std_name, variants in channel_mapping.items():
    # Loop through possible variants for each standard name
    for variant in variants:
        # Make comparison case-insensitive
        matched_channels = [ch for ch in edf_item.ch_names if ch.lower() == variant.lower()]
        print(matched_channels)
        if matched_channels:
            # Rename the first matched variant to standardized name and add to ordered list
            mne.rename_channels(edf_item.info, {matched_channels[0]: std_name})
            ordered_channels.append(std_name)
            break  # Stop after finding the first match for each standard name

ordered_channels
# # Reorder channels based on `ordered_channels`
edf_item.pick_channels(ordered_channels)  # This will keep only the ordered channels

# # Split data into 30-second epochs
events = mne.make_fixed_length_events(edf_item, duration=30)
epochs = mne.Epochs(edf_item, events, tmin=0, tmax=30, baseline=None, preload=True)


In [ ]:
epochs

<Epochs | 983 events (all good), 0 – 30 s (baseline off), ~270.0 MB, data loaded,
 '1': 983>

In [ ]:
edf_item.ch_names

['Resp nasal',
 'Resp chest',
 'Sound SPL',
 'Pulsewave',
 'Resp abdomen',
 'Position',
 'SaO2 SpO2',
 'Pulse',
 'PTT',
 'ECG',
 'EMG right',
 'EMG left',
 'EEG C4/A1',
 'EEG C3/A2*',
 'EEG F4/A1',
 'EOG left',
 'EOG right',
 'EMG chin center',
 'Thermistor',
 'EEG O2/A1',
 'Sound microphone',
 'Hypnogram',
 'Obstr',
 'BR flow',
 'HR central',
 '_Original-Hypnog']

In [ ]:
import matplotlib.pyplot as plt

edf_data = edf_item.get_data()

print(f'edf_data shape: {edf_data.shape}')
y_label_index = edf_item.ch_names.index('Obstr')

y_label = edf_data[y_label_index]


edf_data shape: (26, 5900000)


In [ ]:
y_label

array([ 5.68264622e-15,  4.91838844e-02,  6.94545649e-15, ...,
        4.86177622e-02,  1.51537232e-14, -4.88989181e-02])

In [ ]:
import numpy as np

rounded_labels = np.round(y_label)


import plotly as py
import plotly.graph_objs as go

# Create the plot
fig = go.Figure(data=go.Scatter(y=y_label, mode='lines'))

# Show the plot
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
np.unique(y_label)

array([ 45.9265689 ,  45.92681463,  45.93080247, ..., 210.56963185,
       210.56984464, 210.57203903])

In [ ]:
# prompt: find the index which equal to hyponogram

import os
import mne
import matplotlib.pyplot as plt
!pip install mne

edfpath = '/content/drive/MyDrive/unimedizinSleepEEGproject/shareLink/edf/'

mainz_edf = os.listdir(edfpath)
print(mainz_edf)

edf_item = mne.io.read_raw_edf(os.path.join(edfpath,mainz_edf[0]))
edf_item.annotations



try:
  hypnogram_index = edf_item.ch_names.index('Hypnogram')
  print(f"The index of 'Hypnogram' is: {hypnogram_index}")
except ValueError:
  print("Hypnogram channel not found in the edf file.")

In [ ]:
edf_item.ch_names

['Resp nasal',
 'Resp chest',
 'Sound SPL',
 'Pulsewave',
 'Resp abdomen',
 'Position',
 'SaO2 SpO2',
 'Pulse',
 'PTT',
 'ECG',
 'EMG right',
 'EMG left',
 'EEG C4/A1',
 'EEG C3/A2*',
 'EEG F4/A1',
 'EOG left',
 'EOG right',
 'EMG chin center',
 'Thermistor',
 'EEG O2/A1',
 'Sound microphone',
 'Hypnogram',
 'Obstr',
 'BR flow',
 'HR central',
 '_Original-Hypnog']

In [ ]:
edfpath = '/content/drive/MyDrive/project_sleep/ISRUC/data/1/'

ISRUC_edf = os.listdir(edfpath)

for file in ISRUC_edf:
    if file.endswith(".rec"):
        edf_filepath = os.path.join(edfpath, file)

        new_filename = file.replace(".rec", ".edf")
        new_edf_filepath = os.path.join(edfpath, new_filename)

        if not os.path.exists(new_edf_filepath):
            # Make a copy of the file and rename it
            with open(edf_filepath, "rb") as f_in, open(new_edf_filepath, "wb") as f_out:
                f_out.write(f_in.read())





In [ ]:
edf_item.annotations

<Annotations | 0 segments>

In [ ]:

edf_item = mne.io.read_raw_edf('/content/drive/MyDrive/project_sleep/ISRUC/data/1/1.edf')
x = edf_item.get_data()


NameError: name 'mne' is not defined

In [ ]:
edf_item.ch_names

['LOC-A2',
 'ROC-A1',
 'F3-A2',
 'C3-A2',
 'O1-A2',
 'F4-A1',
 'C4-A1',
 'O2-A1',
 'X1',
 'X2',
 'X3',
 'X4',
 'X5',
 'X6',
 'DC3',
 'X7',
 'X8',
 'SaO2',
 'DC8']

In [ ]:


# Create the plot
fig = go.Figure(data=go.Scatter(y=x[9,1:5000], mode='lines'))

# Show the plot
fig.show()

In [ ]:
edf_data = edf_item.get_data()
edf_data.shape

(19, 5280000)

In [ ]:
# prompt: how can you load txt file as numpy array: /content/drive/MyDrive/project_sleep/ISRUC/data/1/1.txt'

import numpy as np

# Replace 'your_file.txt' with the actual path to your text file
y_label = -np.loadtxt('/content/drive/MyDrive/project_sleep/ISRUC/data/1/1_1.txt')

# Create the plot
fig = go.Figure(data=go.Scatter(y=y_label, mode='lines'))

# Show the plot
fig.show()

In [ ]:
edf_data.shape

(19, 5280000)

In [ ]:

edf_item = mne.io.read_raw_edf('/content/drive/MyDrive/project_sleep/HAG/data/haaglanden-medisch-centrum-sleep-staging-database-1.1/recordings/SN001.edf')

edf_item.ch_names





Extracting EDF parameters from /content/drive/MyDrive/project_sleep/HAG/data/haaglanden-medisch-centrum-sleep-staging-database-1.1/recordings/SN001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-29-032e12ed177e>:1: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  edf_item = mne.io.read_raw_edf('/content/drive/MyDrive/project_sleep/HAG/data/haaglanden-medisch-centrum-sleep-staging-database-1.1/recordings/SN001.edf')
<ipython-input-29-032e12ed177e>:1: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  edf_item = mne.io.read_raw_edf('/content/drive/MyDrive/project_sleep/HAG/data/haaglanden-medisch-centrum-sleep-staging-database-1.1/recordings/SN001.edf')


['EEG F4-M1',
 'EEG C4-M1',
 'EEG O2-M1',
 'EEG C3-M2',
 'EMG chin',
 'EOG E1-M2',
 'EOG E2-M2',
 'ECG']

In [ ]:
edf_item




<RawEDF | SN001.edf, 8 x 6566400 (25650.0 s), ~13 kB, data not loaded>

In [ ]:
# @title ← Run me

!pip install -U mne
import mne
import time
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
from pprint import pprint

from IPython.display import HTML, display
from tensorflow import keras
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from collections import Counter
%matplotlib inline
import matplotlib.pyplot as plt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 75.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

# mount your googld drive to the following path
drive.mount('/content/drive')

# set the path for edf folder and xls folder
edfpath = '/content/drive/MyDrive/unimedizinSleepEEGproject/shareLink/edf/'
xlspath = '/content/drive/MyDrive/unimedizinSleepEEGproject/shareLink/sheets/'

# print the file names
edf_files = [f for f in listdir(edfpath) if isfile(join(edfpath, f))]
# sort read order
xls_files = sorted([f for f in listdir(xlspath) if f[-3:] == 'xls'])

pprint(edf_files)
pprint(xls_files)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['BERIR003_22.01.2021_14-04-03.edf',
 'BOTDO001_22.01.2021_14-05-05.edf',
 'BRAJE000_22.01.2021_13-51-49.edf',
 'BANSI000_22.01.2021_13-46-47.edf',
 'BERIR001_22.01.2021_14-03-06.edf',
 'BAUHA000_22.01.2021_14-12-16.edf',
 'BOTDO000_22.01.2021_14-02-04.edf',
 'BANSI001_22.01.2021_13-44-43.edf',
 'AL_HA000_22.01.2021_12-33-56.edf',
 'AHMMA000_22.01.2021_12-50-34.edf',
 'AHMMA001_22.01.2021_12-52-51.edf',
 'DZIPI001_22.01.2021_13-40-39.edf',
 'CHRHE002_22.01.2021_13-59-53.edf',
 'FUCRA000_22.01.2021_13-07-26.edf',
 'FRAVI000_22.01.2021_13-55-56.edf',
 'FREPE000_22.01.2021_13-11-43.edf',
 'DZIPI000_22.01.2021_13-37-26.edf',
 'CHRFA001_22.01.2021_14-08-04.edf',
 'CHRFA000_22.01.2021_14-06-08.edf',
 'BUCHA000_22.01.2021_12-43-43.edf',
 'CHRHE001_22.01.2021_13-58-57.edf',
 'BUSHA001_22.01.2021_12-31-21.edf',
 'HAGEL001_22.01.2021_13-53-57.edf',
 'HAGTH000_22.01.202

In [ ]:
# Label reference: 'REM': 64,
#                  'N3': 96,
#                  'N2': 128,
#                  'N1': 160,
#                  'Wake': 192

# Manual settings
info_eeg = {'sample_rate': 200, 'sec_epoch': 30}



# Use mne to read edf
raw_edf = mne.io.read_raw_edf(edfpath+edf_files[0])
# raw_data = raw_edf.get_data()
# channels = raw_edf.ch_names

# # Get labeled sequence
# sequence_sleep = get_sleepSeq(raw_data, channels)

# # Get subj name
# name_edf = edf_files[idx_edf]
# name_subj = name_edf[0:8]

# # Get avg label
# subj_dict[name_subj] = get_avgLabel(sequence_sleep, info_eeg)



NameError: name 'edf_files' is not defined

In [ ]:
class EDFDataLoader:
    def __init__(self, label_channel="Hypnogram"):
        self.label_channel = label_channel
        self.channel_mapping = {
            'EEG F4-M1': ['EEG F4-M1', 'F4-A1'],
            'EEG C4-M1': ['EEG C4-M1', 'C4-A1', 'EEG C4/A1', 'C4-A1*'],
            'EEG O2-M1': ['EEG O2-M1', 'O2-A1', 'EEG O2/A1'],
            'EEG C3-M2': ['EEG C3-M2', 'C3-A2', 'EEG C3/A2', 'EEG C3/A2*'],
            'EOG E1-M2': ['EOG E1-M2', 'LOC-A2', 'EOG left'],
            'EOG E2-M2': ['EOG E2-M2', 'ROC-A1', 'EOG right'],
            'ECG': ['ECG', 'X2'],
            'EMG chin': ['EMG chin', 'EMG chin center', 'X1']
        }
        self.label_mappings = {
            0: "W", 1: "N1", 2: "N2", 3: "N3", 4: "R",
            192: "W", 160: "N1", 128: "N2", 96: "N3", 64: "R"
        }

    def load_data_from_directory(self, directory_path):
        subject_data = {}
        for filename in os.listdir(directory_path):
            if filename.endswith(".edf"):
                subject_path = os.path.join(directory_path, filename)
                raw, labels = self.load_single_file(subject_path)

                if "_" in filename:
                    subject_name = filename.split("_")[0]
                elif "." in filename:
                    subject_name = filename.split(".")[0]
                else:
                    subject_name = filename

                subject_data[subject_name] = {'X': raw, 'Y': labels}
        return subject_data

    def load_single_file(self, subject_path):
        raw = mne.io.read_raw_edf(subject_path, preload=True)
        ordered_channels = self.pick_and_rename_channels(raw)

        labels = None
        if self.label_channel in raw.ch_names:
            labels = self.extract_labels_from_raw(raw)
        else:
            labels = self.load_labels_from_text_file(subject_path)

        return raw, labels

    def pick_and_rename_channels(self, raw):
        ordered_channels = []
        for std_name, variants in self.channel_mapping.items():
            for variant in variants:
                matched_channels = [ch for ch in raw.ch_names if ch.lower() == variant.lower()]
                if matched_channels:
                    mne.rename_channels(raw.info, {matched_channels[0]: std_name})
                    ordered_channels.append(std_name)
                    break

        raw.pick_channels(ordered_channels)
        return ordered_channels

    def extract_labels_from_raw(self, raw):
        label_data = raw.get_data(picks=self.label_channel).squeeze()
        return self.aggregate_labels(label_data)

    def load_labels_from_text_file(self, subject_path):
        subject_name = os.path.basename(subject_path).split('.')[0]
        subject_dir = os.path.dirname(subject_path)

        # Find label files starting with subject_name
        label_files = [f for f in os.listdir(subject_dir) if f.startswith(subject_name) and f.endswith('.txt')]

        if not label_files:
            raise FileNotFoundError(f"No label files found starting with {subject_name} in {subject_dir}.")

        label_file = os.path.join(subject_dir, label_files[0])
        with open(label_file, 'r') as f:
            labels = [float(line.strip()) for line in f.readlines()]

        return self.aggregate_labels(labels)

    def aggregate_labels(self, labels):
        # Assuming 30-second epochs and a sample rate of 256 Hz
        epoch_samples = 100 * 30
        num_epochs = len(labels) // epoch_samples
        return [self.majority_label(labels[i * epoch_samples:(i + 1) * epoch_samples]) for i in range(num_epochs)]

    def majority_label(self, label_chunk):
        return Counter(label_chunk).most_common(1)[0][0]


In [ ]:
# Initialize classes
data_loader = EDFDataLoader()
preprocessor = Preprocessor()

# Load data with preprocessing
epochs, labels = data_loader.load_data("subject.edf", label_source="labels_folder", preprocessor=preprocessor)


Extracting EDF parameters from /content/drive/MyDrive/unimedizinSleepEEGproject/shareLink/edf/AHMMA000_22.01.2021_12-50-34.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [ ]:
# loader = EDFLoader(label_channel="SleepStage")
# data = loader.load_data(eeg_source="path/to/eeg_data_folder", label_source="path/to/labels_folder")

